## Example notebook to run ParaSweeper

### <a id='toc1_1_1_'></a>[Functions to be achieved:](#toc0_)
1. reduce ram used; 
2. reduce code simplicity by building new class; (class: Controller; Path; )
3. save more frequently
4. reduce test time

In [ ]:
# magic lines that avoid re-start 
%load_ext autoreload
%autoreload 2

In [ ]:
# Load modules
import pybamm as pb;import pandas as pd;import numpy as np;
import os, json,openpyxl,traceback,multiprocessing,scipy.optimize,sys,gc
import matplotlib.pyplot as plt;
import pickle,imageio,timeit,random,time, signal
from scipy.io import savemat,loadmat;
from pybamm import constants,exp;import matplotlib as mpl

In [ ]:
str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
str_path_1 = os.path.abspath(
    os.path.join(str_path_0,"wip/Rio_Code/ParaSweeper"))
str_path_1

In [ ]:
# reduce variables that require define outside
On_HPC = False
if On_HPC:
    case_no = int(os.environ["PBS_ARRAY_INDEX"])
    Path_Input = "InputData/" 
    BasicPath_Save=os.getcwd() 
else:
    str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
    str_path_1 = os.path.abspath(
        os.path.join(str_path_0,"wip/Rio_Code/Fun_Upgrade"))
    sys.path.append(str_path_1) 
    
    case_no = 6
    Path_Input = os.path.expanduser(
        "~/EnvPBGEM_NC/SimSave/InputData/") # for Linux
    BasicPath_Save =  os.path.expanduser(
        "~/EnvPBGEM_NC/SimSave/P2_R9_Dim")
# import all functions 
from Fun_Upgrade import Run_One_Case
from Fun_Upgrade import *
from TaskResult import *
from TaskConfig import *
from Get_input import *

In [ ]:
# define class for all inputs and customized settings:
# start to do configuration:
path_config = PathConfig(
    On_HPC=On_HPC, Path_Input=Path_Input,
    BasicPath_Save=BasicPath_Save,
    purpose_i="Full_Exp1235_NC", case_no=case_no,
    rows_per_file= 1,Scan_end_end= 12,
    )
global_config = GlobalConfig(
    On_HPC=On_HPC, Runshort=True, 
    Plot_Exp=True, Timeout=True, Return_Sol=True, 
    Check_Short_Time=True, R_from_GITT=True,
    fs=13, dpi=100, Re_No=0, Timelimit=int(3600*48),
    Timeout_text = 'I timed out'   )

exp_config = ExpConfig()
para_config = ParameterConfig()
model_config = ModelConfig()
expData_config = ExperimentDataConfig(cap_0 = 4.86491)
config = TaskConfig(
    path_config=path_config, global_config=global_config, 
    exp_config=exp_config,para_config=para_config,
    model_config=model_config,expData_config=expData_config)
# Load input file

# normal version
# Para_dict_list = load_combinations_from_csv(config.path_config.Para_file) 

# ! For Dan to test:
path_csv_special_for_test = str_path_1 + "/Bundle_6.csv"
Para_dict_list = load_combinations_from_csv(path_csv_special_for_test) 
pool_no = len(Para_dict_list) # do parallel computing if needed

In [ ]:
Para_dict_list[0]['Mesh list'] = '[5,5,5,60,20]'
Para_dict_list[0]

In [ ]:

# Assuming we only do the first one:
config.get_para_dictionary(Para_dict_list[0])

# config.para_config.Para_dict_i['Exp No.'] = 8

config.get_expconfig()
config.initialize_experiment_text()
config.get_exp_data_config()

In [ ]:
Para_dict_list[0]

### Run the model

In [ ]:
# Run the model, assuming only one case:
if config.global_config.Re_No == 0:
    task_result, config = Run_One_Case(config) 
elif config.global_config.Re_No > 0:
    pass
